#Citation

@ARTICLE{Veeling2018-qh,
  title         = "Rotation Equivariant {CNNs} for Digital Pathology",
  author        = "Veeling, Bastiaan S and Linmans, Jasper and Winkens, Jim and
                   Cohen, Taco and Welling, Max",
  month         =  jun,
  year          =  2018,
  archivePrefix = "arXiv",
  primaryClass  = "cs.CV",
  eprint        = "1806.03962"
}

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import h5py
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [4]:
# Function to load data in batches
def load_data_in_batches(file_path_x, file_path_y, batch_size):
    with h5py.File(file_path_x, 'r') as data_file_x, h5py.File(file_path_y, 'r') as data_file_y:
        data_x = data_file_x['x']
        data_y = data_file_y['y'][:]
        total_samples = len(data_x)
        num_batches = total_samples // batch_size

        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, total_samples)
            yield data_x[start_idx:end_idx], data_y[start_idx:end_idx].reshape(-1)  # Reshape labels


In [5]:
# Load train data in batches
train_batches = load_data_in_batches('/content/drive/My Drive/patch/camelyonpatch_level_2_split_train_x.h5',
                                     '/content/drive/My Drive/patch/camelyonpatch_level_2_split_train_y.h5',
                                     batch_size=65536)


In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),

    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model using the batch generator
for batch_x, batch_y in train_batches:
    # Prepare validation data
    with h5py.File('/content/drive/My Drive/patch/camelyonpatch_level_2_split_valid_x.h5', 'r') as valid_x_file:
        valid_x = valid_x_file['x'][:]
    with h5py.File('/content/drive/My Drive/patch/camelyonpatch_level_2_split_valid_y.h5', 'r') as valid_y_file:
        valid_y = valid_y_file['y'][:].reshape(-1)  # Reshape validation labels

    model.fit(batch_x, batch_y, epochs=7, batch_size=32, validation_data=(valid_x, valid_y))

Epoch 1/7
2048/2048 [==============================] - 43s 14ms/step - loss: 0.5315 - accuracy: 0.7359 - val_loss: 0.5194 - val_accuracy: 0.7521
Epoch 2/7
2048/2048 [==============================] - 24s 12ms/step - loss: 0.4894 - accuracy: 0.7709 - val_loss: 0.5412 - val_accuracy: 0.7455
Epoch 3/7
2048/2048 [==============================] - 26s 13ms/step - loss: 0.5022 - accuracy: 0.7602 - val_loss: 0.5309 - val_accuracy: 0.7567
Epoch 4/7
2048/2048 [==============================] - 26s 13ms/step - loss: 0.4708 - accuracy: 0.7849 - val_loss: 0.4660 - val_accuracy: 0.7730
Epoch 5/7
2048/2048 [==============================] - 27s 13ms/step - loss: 0.4491 - accuracy: 0.7985 - val_loss: 0.4893 - val_accuracy: 0.7888
Epoch 6/7
2048/2048 [==============================] - 27s 13ms/step - loss: 0.4348 - accuracy: 0.8039 - val_loss: 0.4724 - val_accuracy: 0.7794
Epoch 7/7
2048/2048 [==============================] - 26s 13ms/step - loss: 0.4114 - accuracy: 0.8175 - val_loss: 0.4735 - val_ac